In [30]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 25 00:00:26 2022

@author: Administrator
"""

import requests
import json
import pandas as pd 
import math
import csv
from math import pow
import numpy as np
import matplotlib.pyplot as plot
import time
import datetime

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# =============================================================================

# Login controller functions

#Get the token to access vMM information  -- via API
def authentication(username,password,aosip):
    username='apiUser'
    password='x564#kdHrtNb563abcde'
    vMM_aosip='140.118.151.248'
    
    url_login = "https://" + aosip + ":4343/v1/api/login"
    payload_login = 'username=' + username + '&password=' + password
    headers = {'Content-Type': 'application/json'}
    get_uidaruba = requests.post(url_login, data=payload_login, headers=headers, verify=False)

    if get_uidaruba.status_code != 200:
        print('Status:', get_uidaruba.status_code, 'Headers:', get_uidaruba.headers,'Error Response:', get_uidaruba.reason)
        uidaruba = "error"

    else:
        uidaruba = get_uidaruba.json()["_global_result"]['UIDARUBA']
        return uidaruba

#show command
def show_command(aosip,uidaruba,command):
    url_login = 'https://' + aosip + ':4343/v1/configuration/showcommand?command='+command+'&UIDARUBA=' + uidaruba
    aoscookie = dict(SESSION = uidaruba)
    AOS_response = requests.get(url_login, cookies=aoscookie, verify=False)
    
    if AOS_response.status_code != 200:
        print('Status:', AOS_response.status_code, 'Headers:', AOS_response.headers,'Error Response:', AOS_response.reason)
        AOS_response = 'error'

    else:
        AOS_response = AOS_response.json()
        
    return AOS_response

def main_loc(IY, floor):    
    #Get the token to access vMM information  -- via API
    token = authentication(username,password,vMM_aosip)
    df = {}
    
    for ap in IY:
        command = 'show+ap+monitor+ap-list+ap-name+IY_'+ap
        list_ap_database = show_command(vMM_aosip,token,command)
        df[ap] = pd.DataFrame(list_ap_database['Monitored AP Table'])
        df[ap]['curr-rssi'] = pd.to_numeric(df[ap]['curr-rssi'])
        df[ap] = df[ap][(df[ap]['ap-type']!='valid')
                                            &(df[ap]['essid']!='eduroam')
                                            &(df[ap]['essid']!='sensor')
                                            &(df[ap]['essid']!='NTUST-PEAP')
                                            &(df[ap]['essid']!='NTUST-UAM')][['essid','bssid','curr-rssi','ap-type', 'chan']]
        df[ap] = df[ap][(df[ap]['curr-rssi']>0)& (df[ap]['curr-rssi']<60)]

    try:
        ap1_int = df[IY[0]]['bssid']
    except Exception:
        ap1_int = None

    try:
        ap3_int = df[IY[1]]['bssid']
    except Exception:
        ap3_int = None

    try:
        ap5_int = df[IY[2]]['bssid']
    except Exception:
        ap5_int = None

    try:
        ap7_int = df[IY[3]]['bssid']
    except Exception:
        ap7_int = None

    try:
        ap9_int = df[IY[4]]['bssid']
    except Exception:
        ap9_int = None

    # Group all the interfering APs on IY- 1F

    ap13_int = pd.concat([ap1_int,ap3_int]).reset_index(drop=True).drop_duplicates()
    try:
        ap57_int = pd.concat([ap5_int,ap7_int]).reset_index(drop=True).drop_duplicates()
        ap_all_int = pd.concat([ap13_int,ap57_int]).reset_index(drop=True).drop_duplicates()
        ap_all_int = pd.concat([ap_all_int,ap9_int]).reset_index(drop=True).drop_duplicates()
    except Exception:
        ap_all_int = ap1_int.reset_index(drop=True).drop_duplicates()
    ap_all = pd.DataFrame(ap_all_int).reset_index(drop=True)
    ap_all['essid'], ap_all['ap type'], ap_all['channel']= '','',''
    for i in range(len(ap)):
        try:
            ap_all[IY[i][-4:]] = None
        except Exception:
            pass
    ap_all['mon AP number'] = None

    for i in range(len(ap_all)):
        no_ap = 0

        for ap in IY:
            try:
            # Get essid
                ap_all['essid'][i] = list(df[ap][(df[ap]['bssid']==ap_all['bssid'][i])]['essid'])[0]
                ap_all['ap type'][i] = list(df[ap][(df[ap]['bssid']==ap_all['bssid'][i])]['ap-type'])[0]
                ap_all['channel'][i] = list(df[ap][(df[ap]['bssid']==ap_all['bssid'][i])]['chan'])[0]
            # Get rssi
                if df[ap]['bssid'].str.contains(ap_all['bssid'][i]).any():
                    ap_all[ap[-4:]][i] = -list(df[ap][df[ap]['bssid']==ap_all['bssid'][i]]['curr-rssi'])[0] 
                    no_ap+=1
            except Exception:
                pass
        ap_all['mon AP number'][i] = no_ap

    ap_all = ap_all[(ap_all['mon AP number']>0)].sort_values('essid').reset_index(drop=True).drop_duplicates()


    ap_all['xloc'], ap_all['yloc'], ap_all['floor'] = '', '', ''
    ap_all_int = ap_all['bssid']
        
    ap_all['floor'] = floor
        
    return ap_all
    
# # =============================================================================

# Username & password & API

username='apiUser'
password='x564#kdHrtNb563abcde'
vMM_aosip='140.118.151.248'
# =============================================================================

# Login & get data

#IYB1 = ['B1F_AP01', 'B1F_AP03', 'B1F_AP05']
IY = []
# IY.append(['B1F_AP01','B1F_AP03','B1F_AP05'])
IY.append(['1F_AP01','1F_AP03','1F_AP05','1F_AP07'])
IY.append(['2F_AP01','2F_AP03','2F_AP05'])
IY.append(['3F_AP01','3F_AP03','3F_AP05','3F_AP07','3F_AP09'])
IY.append(['4F_AP01','4F_AP03','4F_AP05','4F_AP07'])
IY.append(['5F_AP01','5F_AP03','5F_AP05','5F_AP07','5F_AP09'])
IY.append(['6F_AP01','6F_AP03','6F_AP05','6F_AP07','6F_AP09'])
IY.append(['7F_AP01','7F_AP03','7F_AP05','7F_AP07','7F_AP09'])
IY.append(['8F_AP01','8F_AP03','8F_AP05','8F_AP07','8F_AP09'])
IY.append(['9F_AP01','9F_AP03','9F_AP05','9F_AP07'])
IY.append(['10F_AP01','10F_AP03','10F_AP05','10F_AP07'])
IY.append(['11F_AP01'])

f = []
# f.append('B1F')
f.append('1F')
f.append('2F')
f.append('3F')
f.append('4F')
f.append('5F')
f.append('6F') 
f.append('7F')
f.append('8F')
f.append('9F') 
f.append('10F')
f.append('11F')

# =============================================================================

# Run code

df0 = main_loc(IY[0], f[0])
try:
    df1 = main_loc(IY[1], f[1])
except Exception:
    pass
try:
    df2 = main_loc(IY[2], f[2])
except Exception:
    pass
try:
    df3 = main_loc(IY[3], f[3])
except Exception:
    pass
try:
    df4 = main_loc(IY[4], f[4])
except Exception:
    pass
try:
    df5 = main_loc(IY[5], f[5])
except Exception:
    pass
try:
    df6 = main_loc(IY[6], f[6])
except Exception:
    pass
try:
    df7 = main_loc(IY[7], f[7])
except Exception:
    pass
try:
    df8 = main_loc(IY[8], f[8])
except Exception:
    pass
try:
    df9 = main_loc(IY[9], f[9])
except Exception:
    pass
try:
    df10 = main_loc(IY[10], f[10])
except Exception:
    pass
dfa = df0.append(df1, ignore_index=True).append(df2, ignore_index=True).append(df3, ignore_index=True).append(df4, ignore_index=True).append(df5, ignore_index=True).append(df6, ignore_index=True).append(df7, ignore_index=True).append(df8, ignore_index=True).append(df9, ignore_index=True).append(df10, ignore_index=True)
dfb = dfa
# =============================================================================

# Add datetime (GMT +8) and timestamp

import datetime
from datetime import timedelta

ts = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
ts = datetime.datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%fz")
ts
n = 8
# Subtract 8 hours from datetime object
ts = ts - timedelta(hours=n)
ts_tw_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
ts_tw = datetime.datetime.now()

data_json = json.loads(dfa.to_json(orient='records'))

for i in range(len(data_json)):
    data_json[i]['ts'] = ts 
    data_json[i]['DatetimeStr'] = ts_tw_str
    data_json[i]['Datetime'] = ts_tw

# =============================================================================

# Store json data to MongoDB
from bson.objectid import ObjectId
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
import re

previous_day = datetime.now() - timedelta(days=30) 

client = MongoClient("140.118.70.40",27017)
db = client['WiFi_Dashboard_Data']
col=db["Int_Localization2"]
# col.delete_many({"Datetime": {"$lt": previous_day}})
col.insert_many(data_json)
print('Done!')

# # =============================================================================
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import math

n=2.95 
a=-37

def calc_dist(rssi,a,n):
    cal_d= pow(10,((rssi-a)/(-10*n)))
    return cal_d

def dist_error_3d(x1, y1, z1, x2, y2, z2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return dist

def dist_error_2d(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist 

def trilaterate(distances, positions):
  def residuals(x, distances, positions):
    residuals = []
    for i in range(len(distances)):
      residuals.append(np.linalg.norm(x - positions[i]) - distances[i])
    return residuals
  
  initial_guess = np.mean(positions, axis=0)
  result = least_squares(residuals, initial_guess, args=(distances, positions))
  return result.x

def trilateration_2d_co(distances, positions, name):
    # Calculate the position of the object
    position = trilaterate(distances, positions)
    
    final = [name, position[0], position[1]]
    return final

import pymongo
import pandas as pd
myclient = pymongo.MongoClient("140.118.70.40",27017)
mydb = myclient["WiFi_Dashboard_Data"]
mycol = mydb["Int_Localization2"]

data = pd.DataFrame(list(mycol.find().sort('_id',-1).limit(1)))
data['DatetimeStr'][0]
data = pd.DataFrame(list(mycol.find({'DatetimeStr':data['DatetimeStr'][0]}).sort('_id',-1)))
data = data.reset_index(drop=True).drop_duplicates().drop(columns=['_id'], axis=0)
data['xloc4'], data['yloc4'], data['xloc5'], data['yloc5'] = None, None, None, None
data = data[['bssid','essid','ap type', 'channel', 'mon AP number', 'AP01', 'AP03', 'AP05', 'AP07', 'AP09','xloc', 'yloc', 'xloc4', 'yloc4','xloc5', 'yloc5','floor', 'ts', 'DatetimeStr','Datetime']]
data["AP01"] = data['AP01'].apply(lambda x: calc_dist(x,a,n))
data["AP03"] = data['AP03'].apply(lambda x: calc_dist(x,a,n))
data["AP05"] = data['AP05'].apply(lambda x: calc_dist(x,a,n))
data["AP07"] = data['AP07'].apply(lambda x: calc_dist(x,a,n))
data["AP09"] = data['AP09'].apply(lambda x: calc_dist(x,a,n))
data

loc_iy = {}
loc_iy['1F'] = [(16.84, 13.98), (19.59, 1.51),  (23.90, 9.38), (33.62, 7.42), (None, None)]
loc_iy['2F'] = [(12.07, 6.59), (18.45, 11.97),  (19.55, 4.26), (None, None), (None, None)]
loc_iy['3F'] = [(11.24, 2.41), (17.38, 12.13), (20.50, 3.32), (25.42, 12.97), (31.08, 4.85)]
loc_iy['4F'] = [(16.82, 4.96), (16.22, 12.93), (31.53, 4.70), (26.90, 13.09), (None, None)]
loc_iy['5F'] = [(10.78, 3.11), (13.07, 12.29), (15.89, 3.18), (21.97, 12.21), (27.63, 8.02)]
loc_iy['6F'] = [(10.58, 14.12), (19.17, 14.40), (25.32, 14.18), (13.14, 4.25), (28.51, 6.45)]
loc_iy['7F'] = [(14.09, 3.31), (15.77, 12.25), (20.86, 3.33), (23.01, 12.35), (28.74, 4.07)]
loc_iy['8F'] = [(13.80, 3.22), (14.56, 12.04), (19.88, 3.17), (22.64, 12.11), (31.58, 3.25)]
loc_iy['9F'] = [(13.95, 11.37), (26.43, 4.12), (14.33, 3.80), (27.07, 11.01), (None, None)]
loc_iy['10F'] = [(15.56, 3.99), (13.99, 12.02), (27.53, 3.01), (27.43, 12.32), (None, None)]
loc_iy['11F'] = [(14.14, 11.22), (None, None), (None, None), (None, None), (None, None)]

dfa = []
dfa.append(data[(data['floor'] == '1F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '2F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '3F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '4F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '5F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '6F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '7F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '8F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '9F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '10F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfa.append(data[(data['floor'] == '11F')&(data['mon AP number'] > 2)].reset_index(drop=True).drop_duplicates())
dfc = dfa

for j in range(len(dfa)):
    for i in range(len(dfa[j])):
        # Real AP location
        if dfa[j].iloc[i]['essid'] == None:
            name = '-'
        else:
            name = dfa[j].iloc[i]['essid']

        # Define the positions of the reference points
        positions = loc_iy[dfa[j]['floor'][0]]

        # Define the distances between the object and the reference points
        distances = [dfa[j].iloc[i]['AP01'], dfa[j].iloc[i]['AP03'], dfa[j].iloc[i]['AP05'], dfa[j].iloc[i]['AP07'], dfa[j].iloc[i]['AP09']]

        # trilateration_2d(distances, positions, name)

        #Create dataframe
        df = pd.DataFrame({'positions': positions, 'distances': distances})
        #Sort values
        df = df.sort_values('distances').reset_index(drop=True).drop_duplicates()
        df = df.dropna()
        positions = list(df['positions'])
        distances = list(df['distances'])
        if dfa[j].loc[i]['mon AP number'] == 5:
            final = trilateration_2d_co(distances[:5], positions[:5], name)
            dfa[j].at[i, 'xloc5'] = final[1]
            dfa[j].at[i, 'yloc5'] = final[2]
            final = trilateration_2d_co(distances[:4], positions[:4], name)
            dfa[j].at[i, 'xloc4'] = final[1]
            dfa[j].at[i, 'yloc4'] = final[2]
            final = trilateration_2d_co(distances[:3], positions[:3], name)
            dfa[j].at[i, 'xloc'] = final[1]
            dfa[j].at[i, 'yloc'] = final[2]
        if dfa[j].iloc[i]['mon AP number'] == 4:
            final = trilateration_2d_co(distances[:4], positions[:4], name)
            dfa[j].at[i, 'xloc4'] = final[1]
            dfa[j].at[i, 'yloc4'] = final[2]
            final = trilateration_2d_co(distances[:3], positions[:3], name)
            dfa[j].at[i, 'xloc'] = final[1]
            dfa[j].at[i, 'yloc'] = final[2]
        if dfa[j].loc[i]['mon AP number'] == 3:
            final = trilateration_2d_co(distances[:3], positions[:3], name)
            dfa[j].at[i, 'xloc'] = final[1]
            dfa[j].at[i, 'yloc'] = final[2]
dfa[0]

df_final = dfa[0].append(dfa[1], ignore_index=True).append(dfa[2], ignore_index=True).append(dfa[3], ignore_index=True).append(dfa[4], ignore_index=True).append(dfa[5], ignore_index=True).append(dfa[6], ignore_index=True).append(dfa[7], ignore_index=True).append(dfa[8], ignore_index=True).append(dfa[9], ignore_index=True).append(dfa[10], ignore_index=True)
df_final

data_json = json.loads(df_final.to_json(orient='records'))

# for i in range(len(data_json)):
#     data_json[i]['ts'] = ts 
#     data_json[i]['DatetimeStr'] = ts_tw_str
#     data_json[i]['Datetime'] = ts_tw

# =============================================================================

# Store json data to MongoDB
from bson.objectid import ObjectId
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
import re

previous_day = datetime.now() - timedelta(days=30) 

client = MongoClient("140.118.70.40",27017)
db = client['WiFi_Dashboard_Data']
col=db["Int_Coordinate"]
# col.delete_many({"Datetime": {"$lt": previous_day}})
col.insert_many(data_json)
print('Done!')

Done!
Done!


In [31]:
df_final

,bssid,essid,ap type,channel,mon AP number,AP01,AP03,AP05,AP07,AP09,xloc,yloc,xloc4,yloc4,xloc5,yloc5,floor,ts,DatetimeStr,Datetime
0,7a:45:58:12:c2:7e,None,interfering,44+,3,4.075393,NaN,1.168952,3.486365,NaN,25.254467,10.321652,None,None,None,None,1F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
1,8a:45:58:11:c2:7e,None,interfering,6,4,5.568814,4.406236,0.676875,5.150678,NaN,25.155988,6.78056,23.863226,7.99623,None,None,1F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
2,7a:45:58:11:c2:7e,LouisaOffice,interfering,6,3,4.763938,4.763938,0.676875,NaN,NaN,21.2649,8.532979,None,None,None,None,1F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
3,78:45:58:12:c2:7e,LouisaCoffee,interfering,44+,4,4.406236,3.486365,1.366448,4.075393,NaN,25.468483,6.22585,23.734027,7.755077,None,None,1F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
4,78:45:58:11:c2:7e,LouisaCoffee,suspected-rogue(20%),6,4,5.568814,5.150678,0.731824,5.568814,NaN,21.55496,8.436639,23.861556,8.201437,None,None,1F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,e0:23:ff:8f:d3:f8,OnyxWireless-GUEST,interfering,11,5,4.406236,1.726983,2.359833,1.477378,4.763938,19.01813,9.279197,18.294945,8.50738,20.574808,7.574516,8F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
64,e0:23:ff:8f:d3:40,OnyxWireless-GUEST,interfering,60,3,NaN,NaN,5.150678,1.366448,2.359833,25.699195,7.208515,None,None,None,None,8F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
65,e0:23:ff:8f:d3:20,OnyxWireless-GUEST,interfering,6,4,1.477378,1.867181,3.769391,2.982471,NaN,16.711963,8.506599,17.107948,7.900664,None,None,8F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655
66,a8:5e:45:af:a1:54,ASUS_QE_AX5G,interfering,161E,5,0.855467,3.224591,1.726983,5.568814,5.150678,16.027467,5.612388,19.239585,5.493343,19.49241,5.896924,8F,2023-02-04 05:58:59,2023-02-04 13:58:59,2023-02-04 13:58:59.655


In [32]:
dfb

,bssid,essid,ap type,channel,AP01,AP03,AP05,AP07,mon AP number,xloc,yloc,floor,AP09
0,a0:2e:f3:01:d2:0a,5052012052,interfering,1+,-39,None,None,None,1,,,1F,NaN
1,a0:2e:f3:01:9d:69,5052013132,interfering,10-,-43,None,-39,None,2,,,1F,NaN
2,4c:c5:3e:9c:cc:4d,ABC,interfering,1,-53,None,-55,None,2,,,1F,NaN
3,88:de:7c:62:9d:71,IT_SET1,interfering,11,-41,None,None,None,1,,,1F,NaN
4,92:de:7c:62:9d:71,IT_SET2,interfering,11,-44,None,None,None,1,,,1F,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,d2:e8:68:4f:a0:5f,MD-laptop,interfering,1,-43,None,None,None,1,,,10F,NaN
112,ec:ad:e0:cd:bc:23,HEY BOY,interfering,48E,-41,None,None,None,1,,,11F,NaN
113,ec:ad:e0:cd:bc:22,HEY BOY,interfering,10,-43,None,None,None,1,,,11F,NaN
114,40:ee:15:8f:51:18,Prof_Wang_office,interfering,11-,-47,None,None,None,1,,,11F,NaN
